In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
import zarr
import glob
import s3fs
import json
import fsspec

In [2]:
from dask.distributed import Client
from dask_kubernetes import KubeCluster

In [3]:
cluster = KubeCluster()
cluster.adapt(minimum=4, maximum=10, interval='2s', wait_count=3)
client = Client(cluster)
cluster

In [ ]:
# client.close()
# cluster.close()

In [69]:
%%time
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True)

CPU times: user 7.72 s, sys: 392 ms, total: 8.11 s
Wall time: 44.5 s


In [71]:
ds_sst

,Array,Chunk
Bytes,16.70 TB,129.53 MB
Shape,"(6443, 17999, 36000)","(5, 1799, 3600)"
Count,141791 Tasks,141790 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,16.70 TB,129.53 MB
Shape,"(6443, 17999, 36000)","(5, 1799, 3600)"
Count,141791 Tasks,141790 Chunks
Type,float32,numpy.ndarray


In [13]:
%%time
s3 = s3fs.S3FileSystem(
    anon=False,
    key='xxx',    
    secret='xxx',
    client_kwargs=dict(region_name='us-west-2')
)

CPU times: user 52.9 ms, sys: 4.04 ms, total: 57 ms
Wall time: 55.2 ms


# Creating the initial store

### Create a subset and wipe the encoding from the existing store

In [ ]:
# ds_subset = ds_sst.sel(time=slice('2002-06-01', '2002-07-30'))
# ds_subset.analysed_sst.encoding = {}
# ds_subset.analysis_error.encoding = {}
# ds_subset.mask.encoding = {}
# ds_subset.sea_ice_fraction.encoding = {}
# ds_subset

### Initiate the store `('mode' = 'w')`

In [ ]:
# %%time

# chunks = {'time': 60, 'lat': 1799, 'lon': 3600}

# chunk_string = '_'.join(map(str, chunks.values()))

# ds_rechunked = ds_subset.chunk(chunks)
# rechunk_location = f'mur-sst/zarr-{chunk_string}'
# print(f'writing to {rechunk_location}')

# s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
# args = {'consolidated': True, 'mode': 'w'}
# ds_rechunked.to_zarr(s3store, **args)

### Check the new store

In [ ]:
# %%time

# file_location = f's3://mur-sst/zarr-{chunk_string}'
# ds_rechunked = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True, mask_and_scale=False)
# ds_rechunked

# Appending

### Check store

In [82]:
%%time
chunks = {'time': 60, 'lat': 1799, 'lon': 3600}
chunk_string = '_'.join(map(str, chunks.values()))
rechunk_location = f's3://mur-sst/backups/3/zarr-{chunk_string}'
ds_rechunked = xr.open_zarr(fsspec.get_mapper(rechunk_location, anon=True), consolidated=True, mask_and_scale=False)
ds_rechunked

CPU times: user 6.89 s, sys: 243 ms, total: 7.13 s
Wall time: 24.6 s


,Array,Chunk
Bytes,8.32 TB,777.17 MB
Shape,"(6420, 17999, 36000)","(60, 1799, 3600)"
Count,11771 Tasks,11770 Chunks
Type,int16,numpy.ndarray
,Array,Chunk
Bytes,8.32 TB,777.17 MB
Shape,"(6420, 17999, 36000)","(60, 1799, 3600)"
Count,11771 Tasks,11770 Chunks
Type,int16,numpy.ndarray
,Array,Chunk


### Subselect time slice to append

In [78]:
ds_subset = ds_sst.sel(time=slice('2019-05-03','2019-12-28'))
ds_subset.analysed_sst.encoding = {}
ds_subset.analysis_error.encoding = {}
ds_subset.mask.encoding = {}
ds_subset.sea_ice_fraction.encoding = {}
ds_subset

,Array,Chunk
Bytes,622.05 GB,129.53 MB
Shape,"(240, 17999, 36000)","(5, 1799, 3600)"
Count,147071 Tasks,5280 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,622.05 GB,129.53 MB
Shape,"(240, 17999, 36000)","(5, 1799, 3600)"
Count,147071 Tasks,5280 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [79]:
s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
print(f'appending to {rechunk_location}')
args = {'consolidated': True, 'mode': 'a', 'append_dim': 'time'}

appending to s3://mur-sst/backups/3/zarr-60_1799_3600


In [80]:
%%time
# Check the next day we will append is the next day in the year
ds_target = ds_rechunked
ds_source = ds_subset

next_day_target = ds_target.time[-1].values + np.timedelta64(1, 'D')
next_day_target_str = str(next_day_target)[0:10].replace('-', '')
#print(next_day_target_str)

next_day_source = ds_source.time[0].values
next_day_source_str = str(next_day_source)[0:10].replace('-', '') 
#print(next_day_source_str)
    
if not (next_day_target_str == next_day_source_str):
    raise Exception("starting file is not the next day of the year")
elif not (ds_source.dims['time'] % 60 == 0):
    # Check we are only appending if we can chunk to 60
    raise Exception("time span not divisible by 60")
else:
    ds_to_append = ds_source.chunk(chunks)
    ds_to_append.to_zarr(s3store, **args)

CPU times: user 5min 36s, sys: 9.53 s, total: 5min 46s
Wall time: 19min 19s


# Performance + Integrity Testing

In [81]:
%%time
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True, mask_and_scale=False)
ds_subset = ds_sst.sel(time=slice('2002-06-01', '2019-12-28'))
ds_subset

CPU times: user 7.89 s, sys: 411 ms, total: 8.3 s
Wall time: 43.1 s


,Array,Chunk
Bytes,8.32 TB,64.76 MB
Shape,"(6420, 17999, 36000)","(5, 1799, 3600)"
Count,283031 Tasks,141240 Chunks
Type,int16,numpy.ndarray
,Array,Chunk
Bytes,8.32 TB,64.76 MB
Shape,"(6420, 17999, 36000)","(5, 1799, 3600)"
Count,283031 Tasks,141240 Chunks
Type,int16,numpy.ndarray


In [85]:
%%time
#print(ds_subset.analysed_sst.sel(time=slice('2019-04-02', '2019-12-28')).mean().values)
# print()
print(ds_subset.analysed_sst.sel(time='2003-03-01',lat=slice(20,50)).mean().values)

-19588.66883409234
CPU times: user 169 ms, sys: 22.7 ms, total: 191 ms
Wall time: 1.7 s


In [86]:
%%time
#print(ds_rechunked.analysed_sst.sel(time=slice('2019-04-02', '2019-12-28')).mean().values)
# print()
print(ds_rechunked.analysed_sst.sel(time='2003-03-01',lat=slice(20,50)).mean().values)

-19588.66883409234
CPU times: user 3.95 s, sys: 103 ms, total: 4.06 s
Wall time: 33.2 s


In [ ]:
# ds_rechunked = ds_rechunked.isel(time=range(0,4740))
# rechunk_location = f'mur-sst/backups/3/zarr-{chunk_string}'
# print(f'writing to {rechunk_location}')

# s3store = s3fs.S3Map(root=rechunk_location, s3=s3, create=True)
# args = {'consolidated': True, 'mode': 'w'}
# ds_rechunked.to_zarr(s3store, **args)